<a href="https://colab.research.google.com/github/oii-nasif/GlyStruct/blob/master/Glycation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Remove Warnings
import warnings
warnings.filterwarnings(action='ignore',)

# Core:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Machine Learning Algorithms:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

# Dataset Handle:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Performance:
from sklearn.metrics import (confusion_matrix, accuracy_score, classification_report)

In [2]:
# Dataset Link: https://raw.githubusercontent.com/oii-nasif/GlyStruct/master/Glycation.elm
inputFile = '/content/drive/My Drive/Projects/Glycation/DataSet/Glycation.elm'


## Data **Preprocessing**

### Dataset Understanding

In [3]:
# Feature understanding

file = open(file=inputFile, mode='r', encoding='utf-8')
d = {}
for line in file.readlines():
    v = line.split()[3]
    if v in d:
        d[v] += 1
    else:
        d[v] = 1

print(d)

{'Accession': 1, 'Glycation': 6591}


In [4]:
file = open(file=inputFile, mode='r', encoding="utf-8")
C = 0
Sequences = []
for line in file.readlines():
    if C<10:
        print(line.split())
    else: break
    C += 1

Sequences  

['PLMD', 'ID', 'Uniprot', 'Accession', 'Position', 'Type', 'Sequence', 'Species', 'PMIDs']
['PLMD-5', 'O00141', '41', 'Glycation', 'MTVKTEAAKGTLTYSRMRGMVAILIAFMKQRRMGLNDFIQKIANNSYACKHPEVQSILKISQPQEPELMNANPSPPPSPSQQINLGPSSNPHAKPSDFHFLKVIGKGSFGKVLLARHKAEEVFYAVKVLQKKAILKKKEEKHIMSERNVLLKNVKHPFLVGLHFSFQTADKLYFVLDYINGGELFYHLQRERCFLEPRARFYAAEIASALGYLHSLNIVYRDLKPENILLDSQGHIVLTDFGLCKENIEHNSTTSTFCGTPEYLAPEVLHKQPYDRTVDWWCLGAVLYEMLYGLPPFYSRNTAEMYDNILNKPLQLKPNITNSARHLLEGLLQKDRTKRLGAKDDFMEIKSHVFFSLINWDDLINKKITPPFNPNVSGPNDLRHFDPEFTEEPVPNSIGKSPDSVLVTASVKEAAEAFLGFSYAPPTDSFL', 'Homo', 'sapiens', '21612289']
['PLMD-17', 'O00186', '19', 'Glycation', 'MAPPVAERGLKSVVWQKIKATVFDDCKKEGEWKIMLLDEFTTKLLASCCKMTDLLEEGITVVENIYKNREPVRQMKALYFITPTSKSVDCFLHDFASKSENKYKAAYIYFTDFCPDNLFNKIKASCSKSIRRCKEINISFIPHESQVYTLDVPDAFYYCYSPDPGNAKGKDAIMETMADQIVTVCATLDENPGVRYKSKPLDNASKLAQLVEKKLEDYYKIDEKSLIKGKTHSQLLIIDRGFDPVSTVLHELTFQAMAYDLLPIENDTYKYKTDGKEKEAILEEEDDLWVRIRHRHIAVVLEEIPKLMKEISSTKKATEGKTSLSALTQLMKKMPHFRKQITKQVVHLNLAEDCMNKFKLN

[]

#### Sequence Extracting

In [5]:
file = open(file=inputFile, mode='r', encoding="utf-8")

Sequences = []
for line in file.readlines():
    Sequences.append(line.split()[4])

Sequences = Sequences[1:]
Sequences

['...',
 ...]

### Feature Extraction

#### Frequency Count

In [6]:
X = []
for seq in Sequences:
    X.append([seq.count('A'), seq.count('C'), seq.count('D'), seq.count('E'), seq.count('F'), seq.count('G'), seq.count('H'), seq.count('I'), seq.count('K'), seq.count('L'), seq.count('M'), seq.count('N'), seq.count('P'), seq.count('Q'), seq.count('R'), seq.count('S'), seq.count('T'), seq.count('V'), seq.count('W'), seq.count('Y')])

X = np.array(X)

In [7]:
X.shape

(6591, 20)

#### One Hot Encoder

In [8]:
d = {
    'A' : [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'C' : [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'D' : [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'E' : [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'F' : [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'G' : [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'H' : [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'I' : [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'K' : [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'L' : [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'M' : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'N' : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    'P' : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    'Q' : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    'R' : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    'S' : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    'T' : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    'V' : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    'W' : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    'Y' : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
}

In [9]:
# X = []
# for seq in Sequences:
#     x = []
#     for s in seq[:11]:
#         x.append(d[s])
#     x = np.array(x)
#     x = x.flatten()
#     X.append(x)
# X = np.array(X)

#### BLOSUM62

In [10]:
d = {
    'A' : [ 4,  0, -2, -1, -2,  0, -2, -1, -1, -1, -1, -2, -1, -1, -1,  1,  0,  0, -3, -2],
    'C' : [ 0,  9, -3, -4, -2, -3, -3, -1, -3, -1, -1, -3, -3, -3, -3, -1, -1, -1, -2, -2],
    'D' : [-2, -3,  6,  2, -3, -1, -1, -3, -1, -4, -3,  1, -1,  0, -2,  0, -1, -3, -4, -3],
    'E' : [-1, -4,  2,  5, -3, -2,  0, -3,  1, -3, -2,  0, -1,  2,  0,  0, -1, -2, -3, -2],
    'F' : [-2, -2, -3, -3,  6, -3, -1,  0, -3,  0,  0, -3, -4, -3, -3, -2, -2, -1,  1,  3],
    'G' : [ 0, -3, -1, -2, -3,  6, -2, -4, -2, -4, -3,  0, -2, -2, -2,  0, -2, -3, -2, -3],
    'H' : [-2, -3, -1,  0, -1, -2,  8, -3, -1, -3, -2,  1, -2,  0,  0, -1, -2, -3, -2,  2],
    'I' : [-1, -1, -3, -3,  0, -4, -3,  4, -3,  2,  1, -3, -3, -3, -3, -2, -1,  3, -3, -1],
    'K' : [-1, -3, -1,  1, -3, -2, -1, -3,  5, -2, -1,  0, -1,  1,  2,  0, -1, -2, -3, -2],
    'L' : [-1, -1, -4, -3,  0, -4, -3,  2, -2,  4,  2, -3, -3, -2, -2, -2, -1,  1, -2, -1],
    'M' : [-1, -1, -3, -2,  0, -3, -2,  1, -1,  2,  5, -2, -2,  0, -1, -1, -1,  1, -1, -1],
    'N' : [-2, -3,  1,  0, -3,  0,  1, -3,  0, -3, -2,  6, -2,  0,  0,  1,  0, -3, -4, -2],
    'P' : [-1, -3, -1, -1, -4, -2, -2, -3, -1, -3, -2, -2,  7, -1, -2, -1, -1, -2, -4, -3],
    'Q' : [-1, -3,  0,  2, -3, -2,  0, -3,  1, -2,  0,  0, -1,  5,  1,  0, -1, -2, -2, -1],
    'R' : [-1, -3, -2,  0, -3, -2,  0, -3,  2, -2, -1,  0, -2,  1,  5, -1, -1, -3, -3, -2],
    'S' : [ 1, -1,  0,  0, -2,  0, -1, -2,  0, -2, -1,  1, -1,  0, -1,  4,  1, -2, -3, -2],
    'T' : [ 0, -1, -1, -1, -2, -2, -2, -1, -1, -1, -1,  0, -1, -1, -1,  1,  5,  0, -2, -2],
    'V' : [ 0, -1, -3, -2, -1, -3, -3,  3, -2,  1,  1, -3, -2, -2, -3, -2,  0,  4, -3, -1],
    'W' : [-3, -2, -4, -3,  1, -2, -2, -3, -3, -2, -1, -4, -4, -2, -3, -3, -2, -3, 11,  2],
    'Y' : [-2, -2, -3, -2,  3, -3,  2, -1, -2, -1, -1, -2, -3, -1, -2, -2, -2, -1,  2,  7],
}


In [11]:
# X = []
# for seq in Sequences:
#     x = []
#     for s in seq[:11]:
#         x.append(d[s])
#     #end-for
#     x = np.array(x)
#     x = x.flatten()
#     X.append(x)
# X = np.array(X)